<a href="https://colab.research.google.com/github/vincentp3/supersigma/blob/main/sigmadb_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de données à partir de fichiers sigma.

## Chargement des données sigma

In [1]:
!wget https://github.com/SigmaHQ/sigma/archive/refs/heads/master.zip
!unzip master.zip

--2021-11-25 08:24:18--  https://github.com/SigmaHQ/sigma/archive/refs/heads/master.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/SigmaHQ/sigma/zip/refs/heads/master [following]
--2021-11-25 08:24:18--  https://codeload.github.com/SigmaHQ/sigma/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ]   4.68M  9.34MB/s    in 0.5s    

2021-11-25 08:24:19 (9.34 MB/s) - ‘master.zip’ saved [4906864]

Archive:  master.zip
6f87ca8b549ab69ffe039339a9b4a28e9ece2137
   creating: sigma-master/
   creating: sigma-master/.github/
   creating: sigma-master/

## Lecture des yaml et prise en compte

In [2]:
import os
import yaml
from pprint import pprint
def read_recursively():
  PATH=os.getcwd()+"/sigma-master/rules"
  result = [os.path.join(dp, f) for dp, dn, filenames in os.walk(PATH) for f in filenames if os.path.splitext(f)[1] == '.yml']
  return result
def get_yaml(yaml_file):
  with open(yaml_file, "r") as stream:
      try:
        pass
          #print(yaml.safe_load(stream))
      except yaml.YAMLError as exc:
          print(exc)

def get_logsources(dict_yml,yml_file):
  """Construction d'un logsourcen :
    #fieldName
    #value
    #document
  """
  res=[]
  dict_yml = dict_yml["logsource"]
  myres  = {"category" : "", "product" : "", "service" : "", "document" : yml_file}
  for key in dict_yml.keys():
    myres[key]=dict_yml[key]
    res.append(myres)

  return res

def get_selections(dict_yml, yml_file):
  """Construction d'une sélection :
    #fieldName
    #value
    #document
  """
  res=[]
  dict_yml = dict_yml["detection"]
  for pkey in dict_yml.keys():
    
    if pkey != "condition":
      if type(dict_yml[pkey])==dict :
        for key in dict_yml[pkey].keys():
          res.append({"fieldName" : key, "value" : str(dict_yml[pkey][key]), "document" : yml_file})

  return res

def get_all(all_yml):
  logsources=[]
  selections=[]
  for yml_file in all_yml :
      with open(yml_file, "r") as stream:
        dict_yml = yaml.safe_load(stream)
        logsources+=get_logsources(dict_yml,yml_file)
        selections+=get_selections(dict_yml,yml_file)
  return logsources, selections



## Prise en compte sqlite

In [3]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def create_selection(conn, selection):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO selections(fieldName, value,document)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, selection)
    conn.commit()

    return cur.lastrowid

def create_logsource(conn, logsource):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO logsources(category, product,service, document)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, logsource)
    conn.commit()

    return cur.lastrowid

def select_all_selections(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM selections")

    rows = cur.fetchall()

    for row in rows:
        print(row)

def exec_selection(conn, instr):
  cur = conn.cursor()
  cur.execute(instr)

  rows = cur.fetchall()


  return rows

def select_selections_by_fieldName(conn, fieldName):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM selections WHERE fieldName=?",(fieldName,))

    rows = cur.fetchall()

    for row in rows:
        print(row)

def add_data(conn, logsources, selections):
  # create logsources and selections tables
  for logsrc in logsources :
    create_logsource(conn, [logsrc["category"], logsrc["product"], logsrc["service"], logsrc["document"]])
  for selection in selections:
    create_selection(conn, [selection["fieldName"], selection["value"], selection["document"]])

def create_db():
  database = r"pythonsqlite.db"

  import os
  if os.path.isfile(database):
    os.remove(database)

  sql_create_logsources_table = """ CREATE TABLE IF NOT EXISTS logsources (
                                      id integer PRIMARY KEY,
                                      category text NOT NULL,
                                      product text,
                                      service text,
                                      document text
                                  ); """

  sql_create_selections_table = """ CREATE TABLE IF NOT EXISTS selections (
                                      id integer PRIMARY KEY,
                                      fieldName text NOT NULL,
                                      value text,
                                      document text
                                  ); """


  
  # create a database connection
  conn = create_connection(database)
  create_table(conn, sql_create_logsources_table)
  create_table(conn, sql_create_selections_table)
  return conn



In [4]:
all_yml = read_recursively()
logsources, selections  = get_all(all_yml)
print(logsources[0], selections[0])
conn = create_db()
add_data(conn, logsources, selections)

{'category': 'authentication', 'product': '', 'service': '', 'document': '/content/sigma-master/rules/generic/generic_brute_force.yml'} {'fieldName': 'action', 'value': 'failure', 'document': '/content/sigma-master/rules/generic/generic_brute_force.yml'}


## See result of "eventID"

In [5]:
select_selections_by_fieldName(conn,"EventID")

(263, 'EventID', '[12, 13]', '/content/sigma-master/rules/windows/malware/registry_event_mal_azorult.yml')
(387, 'EventID', '[19, 20, 21]', '/content/sigma-master/rules/windows/wmi_event/sysmon_wmi_event_subscription.yml')
(563, 'EventID', '[6, 8]', '/content/sigma-master/rules/windows/other/win_exchange_cve_2021_42321.yml')
(564, 'EventID', '[1006, 1116, 1015, 1117]', '/content/sigma-master/rules/windows/other/win_defender_threat.yml')
(565, 'EventID', '5861', '/content/sigma-master/rules/windows/other/win_wmi_persistence.yml')
(566, 'EventID', '5859', '/content/sigma-master/rules/windows/other/win_wmi_persistence.yml')
(567, 'EventID', '[5001, 5010, 5012, 5101]', '/content/sigma-master/rules/windows/other/win_defender_disabled.yml')
(568, 'EventID', '4662', '/content/sigma-master/rules/windows/other/win_security_wmi_persistence.yml')
(571, 'EventID', '1121', '/content/sigma-master/rules/windows/other/win_defender_psexec_wmi_asr.yml')
(573, 'EventID', '7045', '/content/sigma-master/ru

## Search term

In [6]:
def search_docs_by_term_in_column(conn, table, column,search_str ):
  instr = f"SELECT {column}, document FROM {table} WHERE {column} LIKE '%{search_str}%'"
  res  = exec_selection(conn, instr)
  return res

def search_docs_by_fixed_table(conn, table_fixed, column_fixed, search_str_fixed, table, column, search_str):
  instr_pre = f"SELECT document FROM {table_fixed} WHERE {column_fixed} LIKE '%{search_str_fixed}%'"
  instr = f"SELECT document FROM {table} WHERE {column} LIKE '%{search_str}%' AND document IN ({instr_pre})"
  res  = exec_selection(conn, instr)
  return [a[0] for a in res]
print(search_docs_by_term_in_column(conn, "logsources","product", "a" ))
print("---------------------")
print(search_docs_by_fixed_table(conn, "logsources", "product", "win", "selections", "fieldName", "event"))

[('django', '/content/sigma-master/rules/application/appframework_django_exceptions.yml'), ('django', '/content/sigma-master/rules/application/appframework_django_exceptions.yml'), ('ruby_on_rails', '/content/sigma-master/rules/application/appframework_ruby_on_rails_exceptions.yml'), ('ruby_on_rails', '/content/sigma-master/rules/application/appframework_ruby_on_rails_exceptions.yml'), ('zoho_manageengine', '/content/sigma-master/rules/web/web_cve_2021_40539_manageengine_adselfservice_exploit.yml'), ('zoho_manageengine', '/content/sigma-master/rules/web/web_cve_2021_40539_manageengine_adselfservice_exploit.yml'), ('zoho_manageengine', '/content/sigma-master/rules/web/web_cve_2021_40539_manageengine_adselfservice_exploit.yml'), ('apache', '/content/sigma-master/rules/web/web_nginx_core_dump.yml'), ('apache', '/content/sigma-master/rules/web/web_apache_segfault.yml'), ('apache', '/content/sigma-master/rules/web/web_apache_threading_error.yml'), ('antivirus', '/content/sigma-master/rules/

In [7]:
column = "product"

search_str = ""
instr_prepre = f"SELECT * FROM logsources WHERE {column} LIKE '%{search_str}%'"
print(exec_selection(conn, instr_prepre))
instr_pre = f"SELECT document FROM ({instr_prepre})"
all_docs = exec_selection(conn,instr_pre)

#print(all_docs)
table  = "selections"
column = "fieldName"
search_str = "event"
instr = f"SELECT {column} FROM {table} WHERE {column} LIKE '%{search_str}%' AND document IN ({instr_pre}) GROUP BY {column}"
print("---------------------")
print(instr)
res  = exec_selection(conn,instr)
print(">>> : ",res)

[(1, 'authentication', '', '', '/content/sigma-master/rules/generic/generic_brute_force.yml'), (2, 'application', 'sql', '', '/content/sigma-master/rules/application/app_sqlinjection_errors.yml'), (3, 'application', 'sql', '', '/content/sigma-master/rules/application/app_sqlinjection_errors.yml'), (4, 'application', 'django', '', '/content/sigma-master/rules/application/appframework_django_exceptions.yml'), (5, 'application', 'django', '', '/content/sigma-master/rules/application/appframework_django_exceptions.yml'), (6, 'application', 'python', '', '/content/sigma-master/rules/application/app_python_sql_exceptions.yml'), (7, 'application', 'python', '', '/content/sigma-master/rules/application/app_python_sql_exceptions.yml'), (8, 'application', 'spring', '', '/content/sigma-master/rules/application/appframework_spring_exceptions.yml'), (9, 'application', 'spring', '', '/content/sigma-master/rules/application/appframework_spring_exceptions.yml'), (10, 'application', 'ruby_on_rails', ''

## Heroku

In [8]:
%cd /content/
!python -m install virtualenv

/content
/usr/bin/python3: No module named install


In [14]:
!pip3 install virtualenv
!virtualenv env

     |████████████████████████████████| 5.6 MB 8.9 MB/s 
     |████████████████████████████████| 496 kB 74.5 MB/s 
created virtual environment CPython3.7.12.final.0-64 in 804ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.3.1, setuptools==58.3.0, wheel==0.37.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [26]:
!source /content/env/bin/activate
!/content/env/bin/pip3 install flask gunicorn
!/content/env/bin/pip3 freeze > requirements.txt

In [29]:
!snap install --classic heroku

/bin/bash: snap: command not found


## Flask

### Installation Flask

In [10]:
!pip install flask-ngrok

In [11]:
all_yml = read_recursively()
logsources, selections  = get_all(all_yml)
print(logsources[0], selections[0])
conn = create_db()
add_data(conn, logsources, selections)

{'category': 'authentication', 'product': '', 'service': '', 'document': '/content/sigma-master/rules/generic/generic_brute_force.yml'} {'fieldName': 'action', 'value': 'failure', 'document': '/content/sigma-master/rules/generic/generic_brute_force.yml'}


NameError: ignored

In [ ]:
from flask import Flask, render_template, jsonify, request
from flask_ngrok import run_with_ngrok
from sigma_module import *
app = Flask(__name__)
run_with_ngrok(app)   

@app.route("/")
def home():
  
  print("Launched !")
  return render_template('index.html')
@app.route("/search", methods=['GET'])
def search():
  with sqlite3.connect("pythonsqlite.db") as conn:
    terms = search_docs_by_term_in_column(conn, request.args.get("source"),request.args.get("column"), request.args.get("question"))
  response = jsonify(terms)
  response.headers.add('Access-Control-Allow-Origin', '*')
  return response
@app.route("/getDoc")
def getDoc():
  doc = request.args.get("doc")
  if os.path.isfile(doc):
    print("getDoc : Trouvé !")
  else :
    print("getDoc : Not found !")
  with open(doc, "r") as f:
    res = f.read()
  response = jsonify(res)
  response.headers.add('Access-Control-Allow-Origin', '*')
  return response
app.run()